In [1]:
from utils import marich, dataset
from nets import *
from tqdm import tqdm
import numpy as np
import torch
import pickle
from sklearn.model_selection import train_test_split

Files already downloaded and verified


In [2]:
logreg = LogisticRegression(784,10).cuda()
logreg.load_state_dict(torch.load("./targets/logreg_mnist.pt"))

<All keys matched successfully>

In [3]:
# unlab_x = []
# unlab_y = []
# for j,k in cifar10:
#     j = j.to(device)
#     unlab_x.append(j)
#     unlab_y.append(torch.argmax(logreg(j.view(j.size(0),-1))))

In [4]:
# with open('./imagenet_target/cifar_lr_x.pkl', 'wb') as f:
#   pickle.dump(unlab_x, f)

# with open('./imagenet_target/cifar_lr_y.pkl', 'wb') as f:
#   pickle.dump(unlab_y, f)

In [12]:
with open('./imagenet_target/cifar_lr_x.pkl', 'rb') as f:
  unlab_x = pickle.load(f)

with open('./imagenet_target/cifar_lr_y.pkl', 'rb') as f:
  unlab_y = pickle.load(f)

In [6]:
class basic_dataset(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x

class dataset2(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x
    def get_data(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        return x
    def get_dataset(self, idx):
        x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        return basic_dataset(x,y)
    def get_label(self,idx):
        y = torch.tensor(self.Y)[idx]
        return y
    def get_data_label_loader(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        
        return_set = basic_dataset(x,y)
        return torch.utils.data.DataLoader(return_set, batch_size = 64)

In [7]:
train_id, val_id = train_test_split(range(50000), test_size = 0.2)

In [8]:
unlab_dataset_train = dataset2(torch.stack(unlab_x)[train_id], torch.tensor(unlab_y)[train_id])
unlab_dataset_val = dataset2(torch.stack(unlab_x)[val_id], torch.tensor(unlab_y)[val_id])

In [9]:
testset = datasets.MNIST('../mnist/train/', download=True, train=False, transform=transform)
# trainset, testset = torch.utils.data.random_split(trainset, [50000, 10000])
# valset = datasets.MNIST('../mnist/test/', download=True, train=False, transform=transform)
testloader = DataLoader(testset, batch_size = 256)
validloader = DataLoader(unlab_dataset_val, batch_size = 256)

In [10]:
# kl_marich = []
# dist_marich = []
# agree_marich = []
acc_marich = []

for i in tqdm(range(10)):
  log_attack = LogisticRegression(784,10)
  tl_log, vl_log, tal_log, samp_log = marich(log_attack, unlab_dataset_train, validloader, testloader, budget = 80, init_points = 100, rounds = 10, epochs = 10, LR = 0.02, model_name = "cifar_log"+str(i)+".pt", sampling = "all_egl", device = "cuda", model_type = "log_reg")
  
  acc_marich.append(tal_log)
  # kl_marich.append(kl_log)
  # dist_marich.append(dist)
  # agree_marich.append(agree)
  np.save("./results/acc_marich_lr_cifar.npy", np.array(acc_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/acc_marich.npy", np.array(acc_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/kl_marich.npy", np.array(kl_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/dist_marich.npy", np.array(dist_marich))
  # np.save("/content/drive/MyDrive/Marich/MNIST/kl_dist/agree_marich.npy", np.array(agree_marich))
np.save("./results/samp_lr_cifar.npy", np.array(samp_log))

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\pkpra\AppData\Local\Temp/ipykernel_54172/108530465.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]


Using CUDA


e:\pratik\new_marich\nets.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(outputs)


Test accuracy:  12.91
Training samples:  100
Epoch:  1
Train loss:  2.302399158477783


e:\pratik\new_marich\utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels.append(torch.tensor(l))
c:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

Using gradient sampling on  138  points
Using loss sampling on  112  points


C:\Users\pkpra\AppData\Local\Temp/ipykernel_54172/108530465.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
C:\Users\pkpra\AppData\Local\Temp/ipykernel_54172/108530465.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
e:\pratik\new_marich\nets.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(outputs)
e:\pratik\new_marich\u

Budget:  959
Training samples:  959
Training
Epoch:  1
Train loss:  1.642083994547526
Validation loss:  1.6601160794496537
Epoch time -----  0.8504838943481445  sec
validation loss minimum, saving model
Epoch:  2
Train loss:  1.6274529059727987
Validation loss:  1.6587822318077088
Epoch time -----  0.8995959758758545  sec
validation loss minimum, saving model
Epoch:  3
Train loss:  1.6177971839904786
Validation loss:  1.6555938303470612
Epoch time -----  0.8745460510253906  sec
validation loss minimum, saving model
Epoch:  4
Train loss:  1.6105762004852295
Validation loss:  1.6547764927148818
Epoch time -----  0.8149230480194092  sec
validation loss minimum, saving model
Epoch:  5
Train loss:  1.605657156308492
Validation loss:  1.6516445755958558
Epoch time -----  0.6955840587615967  sec
validation loss minimum, saving model
Epoch:  6
Train loss:  1.6012707471847534
Validation loss:  1.651684957742691
Epoch time -----  0.6150002479553223  sec
Epoch:  7
Train loss:  1.597223416964213
V

100%|██████████| 10/10 [30:04<00:00, 180.43s/it]

Test accuracy:  80.77
